In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [37]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

Download from page data about Toronto neigbourghoods

In [38]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [39]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [40]:
# Get the title  
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [41]:
# Move all table wiki to data 
data = []
my_string = ''
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    #my_string = ''.join(cols)
    #my_list = my_string.split(",")
    data.append([ele for ele in cols if ele]) # Get rid of empty values


Add Beigbourghood via comma separated if the same Postal code and Borough exist

In [42]:
for data_index in range(len(data)):
    if len(data[data_index]) == 3:
        postcode = data[data_index][0]
        borough = data[data_index][1]
        if borough != 'Not assigned':
            neighbourghood = data[data_index][2]
            if neighbourghood == 'Not assigned':
                neighbourghood = borough
        if data_index>0:
            for partial_index in range(data_index):
                if len(data[partial_index]) == 3:
                    if ((data[partial_index][0] == postcode) and (data[partial_index][1] == borough)):
                        data[partial_index][2] = data[partial_index][2] + ', ' + neighbourghood
                        data[data_index][1] = 'Not assigned' # Mark as 'Not assigned' for future remove
                        
  

In [43]:
#Create data frame  aliasing as pd and move from data negbourghoods to df

df = pd.DataFrame()
df = pd.DataFrame(columns=['Postal Code','Borough', 'Neighbourghood'])

for data_index in range(len(data)):
    if len(data[data_index]) == 3:
        postcode = data[data_index][0]
        borough = data[data_index][1]
        if borough != 'Not assigned':
            neighbourghood = data[data_index][2]
            if neighbourghood == 'Not assigned':
                neighbourghood = borough
            #  Append a new value to dataframe
            df = df.append({'Postal Code' : postcode , 'Borough' : borough, 'Neighbourghood' : neighbourghood} , ignore_index=True)
            #print(postcode, borough, neighbourghood)
#df = pd.DataFrame()
#df = pd.DataFrame(table_data, columns=['Postal code','Borough', 'Neighbourghood'])
df


,Postal Code,Borough,Neighbourghood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [44]:
df.shape

(103, 3)

In [45]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2019-06-10 20:26:24--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-06-10 20:26:25--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-10 20:26:27--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

In [46]:
df_csv = pd.read_csv('Geospatial_Coordinates.csv')
df_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
# Add coordinates to 
df_suffix = pd.merge(df, df_csv, on='Postal Code',how='left')

df_suffix

,Postal Code,Borough,Neighbourghood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [53]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you don't have this library installed
import folium # map rendering library
#!conda install -c conda-forge geopy --yes # uncomment this line if you don't have this library installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [58]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_suffix['Latitude'], df_suffix['Longitude'], df_suffix['Borough'], df_suffix['Neighbourghood']):
    label = '{}, {}'.format(df_suffix, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto